In [1]:
%matplotlib inline

In [1]:
# Author: Yulou Zhou
#Weight loss function by commonality (and normalize)
#padded sequence
#for loop to test parameters
#for loop to get prev generated chord


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
from MyDataset import MyDataset
torch.manual_seed(1)

updated


In [52]:
batch_size = 1
train_data = MyDataset('testgeneration', prevchord = 6)
weights = train_data.stats()
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
c=0
for i in data_loader:
    print("p: ", i['p'])
    print("m: ", i['m'])
    #print("mask: ", i['mask'])
    print("l: ", i['l'])
    c += 10
    if c > 0: break

p:  tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]])
m:  tensor([[ 0,  0,  0,  0,  0,  0,  8,  8]])
l:  tensor([[ 1,  4,  7]])


In [4]:
def normalize(array):
    return [i / sum(array) for i in array]
newweights = []

def weightloss(weights, eps):
    for w in weights:
        newweights.append(torch.FloatTensor( normalize([1/(i + eps) for i in w] ) ) )
    #print(newweights)


In [5]:
class LSTMTagger(nn.Module):

    def __init__(self, p_embed_dim, m_embed_dim, p_hidden_dim, m_hidden_dim, vocab_size, tagset_size, batch_size = 1):
        super(LSTMTagger, self).__init__()
        self.batch_size = batch_size
        
        self.p_embed_dim, self.m_embed_dim = p_embed_dim, m_embed_dim
        #refer to https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222/2
        
        self.p_embed = nn.Embedding(vocab_size, p_embed_dim)
        self.m_embed = nn.Embedding(vocab_size, m_embed_dim)
        self.p_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        self.m_embed.weight.data.copy_(torch.from_numpy(np.identity(vocab_size)))
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.p_lstm = nn.LSTM(p_embed_dim, p_hidden_dim, bidirectional = False)
        self.m_lstm = nn.LSTM(m_embed_dim, m_hidden_dim, bidirectional = False)
        

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear((p_hidden_dim + m_hidden_dim) , tagset_size)
        self.root2second = nn.Linear((p_hidden_dim + m_hidden_dim)  + 1, tagset_size)
        self.root2third = nn.Linear((p_hidden_dim + m_hidden_dim)  + 1, tagset_size)
        
        self.p_hidden = self.init_hidden(p_hidden_dim)
        self.m_hidden = self.init_hidden(m_hidden_dim)

    def init_one_hot(self, vocab_size):
        #initialize each embedding
        #stack them together (or other ways to have  pretrained embeddings)
        #pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
        #we should turn data into torch.from_numpy(pretrained_weight)
        #embed.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
        torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
    
    def init_hidden(self, hidden_dim):
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, self.batch_size, hidden_dim),
                torch.zeros(1, self.batch_size, hidden_dim))

    def forward(self, prev, melody, mask):
        
        prev_embeds = self.p_embed(prev)
        x1 = torch.transpose(prev_embeds, 0, 1) #TODO: DEBUG THIS LINE (probably the last batch)
        
        #TODO: ensure that melody input is a tensor
        
        melody_embeds =  self.m_embed(melody) #Error here
        #print(melody_embeds.size())
        x2 = torch.transpose(melody_embeds, 0, 1)[:8]
        
        p_lstm_out, self.p_hidden = self.p_lstm(x1, self.p_hidden)
        m_lstm_out, self.m_hidden = self.m_lstm(x2, self.m_hidden)      
        
        p_fstate, m_fstate = p_lstm_out[-1], m_lstm_out[-1]
        

        concat = torch.cat((p_fstate, m_fstate), 1)
        #print(p_fstate.size(), m_fstate.size())
        tag_space = self.hidden2tag(concat) #REASON: you only need the final state
        tag_scores = F.log_softmax(tag_space, dim = 1)
        

        withroot = torch.cat((concat, tag_scores.max(1)[1].float().view(-1,1)), 1)
        second = self.root2second(withroot)
        third = self.root2third(withroot)
        second_scores = F.log_softmax(second, dim=1)
        third_scores = F.log_softmax(third, dim=1)
        
        #stacked = torch.stack([tag_scores, second_scores, third_scores], dim=0)

        return [tag_scores, second_scores, third_scores]

In [ ]:
import random
for iteration in range(20):

    P_HIDDEN_DIM = max(int(np.random.normal(20, 25)), 5)
    M_HIDDEN_DIM = max(int(np.random.normal(20, 25)), 5)
    vocab_size = 13
    tagset_size = 13
    EPS = random.uniform(0.001, 0.12)
    ITER = max(8000, min(int(np.random.normal(15000, 3000)), 50000))
    P_EMBEDDING_DIM = vocab_size
    M_EMBEDDING_DIM = vocab_size

    print(iteration, " iter: ", ITER, "  phid: ", P_HIDDEN_DIM, "  mhid: ", M_HIDDEN_DIM, "  eps: ", EPS)
    newweishts = weightloss(weights, 0.005)
    model = LSTMTagger(P_EMBEDDING_DIM, M_EMBEDDING_DIM, P_HIDDEN_DIM, M_HIDDEN_DIM, vocab_size, tagset_size, batch_size = batch_size)
    loss_function = nn.NLLLoss(weight = newweights[0])
    loss_function2 = nn.NLLLoss(weight = newweights[1])
    loss_function3 = nn.NLLLoss(weight = newweights[2])
    optimizer = optim.SGD(model.parameters(), lr=0.2)
    
    for epoch in range(ITER):  # again, normally you would NOT do 300 epochs, it is toy data
        for i in data_loader:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Also, we need to clear out the hidden state of the LSTM,
            # detaching it from its history on the last instance.
            model.p_hidden = model.init_hidden(P_HIDDEN_DIM)
            model.m_hidden = model.init_hidden(M_HIDDEN_DIM)

            # Step 2. Get our inputs ready for the network, that is, turn them into
            # Tensors of word indices.
            p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
            #print(m)
            tag_scores = model(p, m, mask)
            # Step 3. Run our forward pass.

            # Step 4. Compute the loss, gradients, and update the parameters by
            #  calling optimizer.step()
            #print(targets[:1])
            loss = 0

            loss += loss_function(tag_scores[0], torch.t(l)[0])  
            loss += 0.3 * loss_function2(tag_scores[1], torch.t(l)[1])
            loss += 0.3 * loss_function3(tag_scores[2], torch.t(l)[2])   

            loss.backward()
            optimizer.step()
            break
        if(epoch%5000 == 0): 
            print(epoch, loss)
            accuracy = 0.0
            c = 0
            for j in range(2):
                for i in data_loader:
                    c+=1
                    p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
                    tag_scores = model(p, m, mask)
                    pred = tag_scores[0].max(1)[1].numpy()
                    truth = torch.t(l)[0].numpy()
                    print(pred[:20], truth[:20])
                    accuracy += np.average(pred == truth)
                    if (c>=1): break
            print("accuracy:", accuracy/c)

    accuracy = 0.0
    c = 0

    for j in range(10):
        for i in data_loader:
            c+=1
            p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
            tag_scores = model(p, m, mask)
            pred = tag_scores[0].max(1)[1].numpy()
            truth = torch.t(l)[0].numpy()
            if(c<4): print(pred[:20], truth[:20])
            accuracy += np.average(pred == truth)
            if (c>=1): break
    print("accuracy:", accuracy/c)
    print(str(int(accuracy/c * 1000)))
    
    torch.save(model, "/mnt/d/230/2-Iterations-lstm" + str(iteration)+"_"+str(int(accuracy/c * 1000)))

0  iter:  12324   phid:  5   mhid:  40   eps:  0.029377359640038608
0 tensor(4.0761)
[11 11 11 11 12 12 11 11 12 12 11 11 12 12 12 11 12 11 11 11] [ 5  1  3  6  1  6  1  2 11 10  2  6  8  1  8  9  3  6 11  6]
[12 11 12 12 11 11 12 11 12 12 12 11 12 12 11 12 12 12 11 11] [ 3  3  8  5 11 10  8  8  8 10  6  1 12 11 12 11  3  1 11  6]
accuracy: 0.064453125
5000 tensor(3.0368)
[11  1  3  8  3  2  0  3  1  1  3  4  4  1  6  9  7  1  4  1] [11  8 10  1  1  6  1  5  1  6 10  8  4  1  3  3  3  8  6  5]
[11  1 11  1  1  1  1 11  1  6  4  2  1  5  1  4 12  1  1  6] [11  1  6  1  1  8  1  1  6  5  1  3  1 10 10  6 12  3  3  1]
accuracy: 0.26171875
10000 tensor(2.4807)
[ 1  4  3  6  4  8  1 11  0 10  8  6  2  1  4  3  8  4  1  4] [ 3  6  6  1  6  1  8  4  3 10  6  1  9  3  9  8  1  1  1  4]
[ 3  3 12  6  2 10  8  1  1  1  3  5  6  4  6 11 12  9  9  4] [ 1  3 10  9  8 10  3  1  6  8 10  5  1  5 11 11  8  9  6  9]
accuracy: 0.259765625
[ 9 10  8  7 12  1  4  9  1  3 12  1  8  1  6  1 10  1  5  8] [ 8

/home/yuloucn/anaconda3/envs/nlu/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type LSTMTagger. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[12  8  5 12  5 12  5 12  5 10  5  5 12 12 12 10 12 10  6 10] [ 3  4  1  6 11  6 10  5  6  8 11  8  3  4  3  1  3  1  3 11]
[12 12  5  5 12 12  5  6  8  6 12  8 12 12 12  8  5 12 12 12] [ 1  6 12  1  8  9  9  3 11 11  3 11  1  8 10 11  5 10  8  8]
accuracy: 0.0556640625
5000 tensor(2.3476)
[11  2  8  8  8  9  9  1  3  4  3 11  7  7  2  0  7  6  1  1] [ 6  9  3 10 12  1  1  8 10  8  6 11  5  7  2  9  1  6  1  8]
[ 8  6  0  8 12 10 11  3  2  6 11  6  9  1 11  9  1  1 11 11] [ 3  1 12  8  3 10  6  1  4  3 11  6  6  8  1  6  8  1  8  6]
accuracy: 0.3037109375
10000 tensor(2.3232)
[ 7  5  9  9  1  0  4  3  8  8  1  0  1  8 10  1  6  5  6 10] [12  7  6  9 11  6  6  8  8  8  6  6  1  3  5  1 10  5  8  3]
[ 6 12  6  6 11  4  1  9  5  3  5  4  3  9  1  6  6  6  1  6] [ 6  8  6  8  1  1  8 11  5  8  5 11 12  9  6  6  1  1  1  8]
accuracy: 0.3095703125
[ 7  9 12 11  9 10  9  3  2  8 12  6  6  6  4  8  6  4 11 11] [ 1  9  1  6  8  3  5  8  5  3 10  3  3  8 10  6  6  4  3  1]
[ 3  4  9  8  6  6  4 

15000 tensor(1.7439)
[ 6  5 12  8  3  6  6  8  3  3  1  1  4  6 11  9  0  3  0  4] [ 6  1  3 11  8  6  1  8  8  8  1  1  6  8  6  9 11  1  6  5]
[ 1  1  3 11  1  6 11  2 10  6  3  6  6  3  5  4  3 10  8  8] [ 1  1  3  8  3 10  4  6  3  6  8 10  6  7 10  3  3  1  3  6]
accuracy: 0.302734375
[12  7  8  4  6  2  1  6 11  1  6 10  3  3 11  3 11  1 10  9] [12 10  1  1  8  3  1  3 11 10  6  1  6  8 10  8  8  1  0 11]
[ 1  5  5  0 10  9  8  3 11  3  1  6 10  6  6  6  3  8  3 11] [ 2 10  6  1 10  4  6 11  5  3  1  1  3  6  6  1 11  8 10  4]
[ 1 10 11 11  3  9 10  3  8  8  9 10  3  6  1 10  5  2  0  6] [ 8 10  4 11  3  9  5  8  8  8  9  6  6 10  5 10  5  1 11  1]
accuracy: 0.3138671875
313
7  iter:  13648   phid:  5   mhid:  25   eps:  0.003146045623024015
0 tensor(4.3888)
[5 5 9 9 5 5 9 5 5 5 5 5 5 5 5 5 5 5 5 9] [ 6  1  1  1  6  5  2  8 10  6  1 10  8  2  1 10  6 10  8  6]
[9 5 5 9 5 5 5 9 5 5 9 5 5 5 9 5 5 5 5 5] [ 4  6  6  6  8  1 11  8  6  1  7  3 10  1  5  1  3  1  3  1]
accuracy: 0.07031

In [ ]:

accuracy = 0.0
c = 0
for j in range(20):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = model(p, m, mask)
        pred = tag_scores[0].max(1)[1].numpy()
        truth = torch.t(l)[0].numpy()
        print(pred[:20], truth[:20])
        accuracy += np.average(pred==truth)
        if (c>=1): break
print("accuracy:", accuracy/c)

In [69]:
#torch.save(model, "/mnt/d/230/2-Iterations-lstm")

batch_size = 1
train_data = MyDataset('testgeneration', prevchord = 6)
weights = train_data.stats()
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
c=0

model = torch.load("/mnt/d/230/2-Iterations-lstm1_331")
torch.save(model.state_dict(), "../state")
model = LSTMTagger(13, 13, 26, 13, 13, 13, batch_size = 1)
model.load_state_dict(torch.load("../state"))

accuracy = 0.0
c = 0
for j in range(1):
    for i in data_loader:
        c+=1
        p, m, l, mask = i['p'], i['m'], i['l'], i['mask']
        tag_scores = model(p, m, mask)
        pred = np.asarray([tag_scores[0].max(1)[1],tag_scores[1].max(1)[1],tag_scores[2].max(1)[1]])
        truth = np.asarray([torch.t(l)[0], torch.t(l)[1], torch.t(l)[2]])
        print(pred, truth)
        accuracy += np.average(pred==truth)
print("accuracy:", accuracy/c)

[0 0 0] [1 4 7]
[1 4 7] [1 4 7]
[1 4 7] [1 4 7]
[6 4 7] [6 4 7]
[6 4 7] [6 4 7]
[11  5  9] [11  5  9]
[5 5 9] [5 5 9]
[5 3 9] [5 3 9]
[10  3  9] [1 4 7]
accuracy: 0.7777777777777778


Create the model:



Train the model:

